# Relatório Científico - NeuroPasso
## Análise de Alta Resolução (Cinemática e Eletromiografia)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sqlite3
import numpy as np
from scipy.signal import butter, filtfilt

# Configuração Visual
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [14, 16] # Altura maior para 4 gráficos
pd.set_option('display.max_rows', None)

### 1. Seleção do Paciente
Execute a célula abaixo para ver os pacientes e escolha o **ID do Paciente**.

In [ ]:
db_path = '../backend/clinic.db'
conn = sqlite3.connect(db_path)

df_patients = pd.read_sql_query("SELECT * FROM patients", conn)
display(df_patients)

In [ ]:
# === ESCOLHA O PACIENTE ===
PATIENT_ID = 1  # <--- DIGITE O ID DO PACIENTE AQUI
# ==========================

### 2. Seleção da Sessão
Agora veja todas as sessões deste paciente e escolha o **ID da Sessão**.

In [ ]:
query_sessions = f"SELECT id, timestamp, duration_seconds, max_angle_esq, max_angle_dir FROM sessions WHERE patient_id = {PATIENT_ID}"
df_sessions = pd.read_sql_query(query_sessions, conn)
display(df_sessions)

In [ ]:
# === ESCOLHA A SESSÃO ===
SESSION_ID = 10  # <--- DIGITE O ID DA SESSÃO AQUI
# ========================

### 3. Processamento e Plotagem (Dados Brutos)
Abaixo estão os 3 gráficos com os dados originais (Raw Data).

In [ ]:
def get_session_raw_data(session_id):
    try:
        query = f"SELECT raw_data_blob, duration_seconds FROM sessions WHERE id = {session_id}"
        row = pd.read_sql_query(query, conn).iloc[0]
        raw_json = row['raw_data_blob']
        duration = row['duration_seconds']
        data = json.loads(raw_json)
        return pd.DataFrame(data), duration
    except IndexError:
        print(f"Sessão {session_id} não encontrada!")
        return None, None

df_raw, duration = get_session_raw_data(SESSION_ID)

if df_raw is not None and not df_raw.empty:
    # --- Lógica de Tempo de Alta Resolução ---
    num_samples = len(df_raw)
    if duration and duration > 0:
        time_axis = np.linspace(0, duration, num_samples)
    else:
        time_axis = df_raw.index * 0.1
    df_raw['seconds_high_res'] = time_axis

    # --- Plotagem (3 Subplots Separados) ---
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=False, figsize=(14, 14))
    
    # Cores do App:
    # Ângulo: Lilás (#a78bfa)
    # EMG: Azul (#3b82f6)
    # ECG: Marrom/Âmbar (#d97706)

    # 1. Gráfico de Ângulo
    if 'ESQ_angle' in df_raw.columns: 
        ax1.plot(df_raw['seconds_high_res'], df_raw['ESQ_angle'], label='Esq (Parética)', color='#a78bfa', linestyle='--', linewidth=2)
    if 'DIR_angle' in df_raw.columns: 
        ax1.plot(df_raw['seconds_high_res'], df_raw['DIR_angle'], label='Dir (Controle)', color='#a78bfa', linewidth=2)
    ax1.set_title(f'Ângulo da Flexão do Quadril x Tempo - Sessão {SESSION_ID}', fontsize=14)
    ax1.set_ylabel('Ângulo (°)', fontsize=12)
    ax1.set_xlabel('Tempo (segundos)', fontsize=12)
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.set_ylim(0, 180)

    # 2. Gráfico de EMG
    if 'ESQ_emg' in df_raw.columns: 
        ax2.plot(df_raw['seconds_high_res'], df_raw['ESQ_emg'], label='Esq (Parética)', color='#3b82f6', linestyle='--', linewidth=1.5)
    if 'DIR_emg' in df_raw.columns: 
        ax2.plot(df_raw['seconds_high_res'], df_raw['DIR_emg'], label='Dir (Controle)', color='#3b82f6', linewidth=1.5)
    ax2.set_title(f'Sinal EMG bruto x tempo - Sessão {SESSION_ID}', fontsize=14)
    ax2.set_ylabel('Sinal (0-4095)', fontsize=12)
    ax2.set_xlabel('Tempo (segundos)', fontsize=12)
    ax2.legend(loc='upper right')
    ax2.grid(True, linestyle='--', alpha=0.6)

    # 3. Gráfico de ECG
    if 'ESQ_ecg' in df_raw.columns: 
        ax3.plot(df_raw['seconds_high_res'], df_raw['ESQ_ecg'], label='Esq (Parética)', color='#d97706', linestyle='--', linewidth=1.5)
    if 'DIR_ecg' in df_raw.columns: 
        ax3.plot(df_raw['seconds_high_res'], df_raw['DIR_ecg'], label='Dir (Controle)', color='#d97706', linewidth=1.5)
    ax3.set_title(f'Sinal ECG bruto x tempo - Sessão {SESSION_ID}', fontsize=14)
    ax3.set_ylabel('Sinal (0-4095)', fontsize=12)
    ax3.set_xlabel('Tempo (segundos)', fontsize=12)
    ax3.legend(loc='upper right')
    ax3.grid(True, linestyle='--', alpha=0.6)

    plt.subplots_adjust(hspace=0.4) # Espaço entre gráficos
    plt.show()
    
    print(f"Total de amostras: {num_samples}")
    print(f"Duração: {duration}s")
else:
    print("Não há dados válidos nesta sessão.")

### 4. Insight Profissional: Ativação Combinada (EMG + ECG)
No sistema NeuroPasso, utilizamos tanto o sensor de EMG quanto o de ECG para estimar a ativação muscular total.
O gráfico abaixo mostra a **Média Combinada** `(EMG + ECG) / 2` com aplicação de filtro Butterworth para visualizar a envoltória de ativação.

In [ ]:
# Função de Filtro Butterworth
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Parâmetros do Filtro
FS = 100  # Frequência de amostragem estimada (Hz)
CUTOFF = 3 # Frequência de corte (Hz) - 3Hz para uma envoltória bem suave

if df_raw is not None and not df_raw.empty:
    plt.figure(figsize=(14, 6))
    
    # --- Processamento Perna DIREITA (Controle) ---
    if 'DIR_emg' in df_raw.columns and 'DIR_ecg' in df_raw.columns:
        raw_combined_dir = (df_raw['DIR_emg'] + df_raw['DIR_ecg']) / 2
        try:
            filtered_dir = butter_lowpass_filter(raw_combined_dir, CUTOFF, FS)
            plt.plot(df_raw['seconds_high_res'], filtered_dir, 
                     label='Dir (Controle) - Filtrado', 
                     color='#8b5cf6', linewidth=2.5) # Roxo Sólido
        except Exception as e:
            print(f"Erro filtro DIR: {e}")

    # --- Processamento Perna ESQUERDA (Parética) ---
    if 'ESQ_emg' in df_raw.columns and 'ESQ_ecg' in df_raw.columns:
        raw_combined_esq = (df_raw['ESQ_emg'] + df_raw['ESQ_ecg']) / 2
        try:
            filtered_esq = butter_lowpass_filter(raw_combined_esq, CUTOFF, FS)
            plt.plot(df_raw['seconds_high_res'], filtered_esq, 
                     label='Esq (Parética) - Filtrado', 
                     color='#ec4899', linestyle='--', linewidth=2.5) # Rosa Tracejado
        except Exception as e:
            print(f"Erro filtro ESQ: {e}")

    plt.title(f'Ativação Muscular Total (Média EMG + ECG) - Filtrada ({CUTOFF}Hz)', fontsize=14)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude Média (u.a.)')
    plt.legend(loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()
else:
    print("Não há dados para plotar.")